In [1]:
import os
import pandas as pd
import json
import matchms
from pathlib import Path
import py4cytoscape as p4c
from zipfile import ZipFile
import io
from IPython.display import SVG, display
from extractor.network import Network
from extractor.compounds import Compounds
from extractor.manufactured.datadirs import *
os.chdir("../")

In [2]:
os.getcwd()


'/home/olivier/Local/Nextcloud LAMSADE/Recherche/AMCD/Chimie/MS2DECIDE/Data/Extractor'

In [15]:
# compounds = Compounds.from_tsv(GENERATED_DIR_TABLES / "Compounds joined.tsv").df.reset_index().set_index("Semantic id")
compounds = Compounds.from_tsv(GENERATED_DIR_TABLES / "Compounds joined.tsv").df
indicators = compounds.loc[:, ["cg", "cs", "ci", "tgs", "tgi", "tsi", "Rank K"]]
ranks = compounds.loc[:, compounds.columns.str.startswith("Rank")]
compounds

,Chemical name,Reported,Chemical class,InChI,Relative molecular weight,Precursor m/z,Retention time (sec),Precursor m/z − relative molecular weight,Analog InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.02,Analog Smiles GNPS; peaks ≥ 6; Δ mass ≤ 0.02,...,Rank GNPS iterated,Rank min Sirius,Rank max Sirius,Ranks Sirius,Rank min ISDB-LOTUS,Rank max ISDB-LOTUS,Rank ISDB-LOTUS,Rank min K,Rank max K,Rank K
Id,,,,,,,,,,,,,,,,,,,,,
1,13-hydroxylupanine,1,Bisquinolizidine alkaloid,InChI=1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14...,264.369,265.190861,43.819,0.821861,1S/C15H24N2O2/c18-12-4-5-16-8-10-6-11(14(16)7-...,C1C[C@@H]2[C@H]3C[C@@H](CN2C(=O)C1)[C@@H]4C[C@...,...,62,60,60,60,95,95,95,64,64,64
2,3-beta-amino-pregnane,1,Steroidal alkaloid,InChI=1S/C21H37N/c1-4-14-6-8-18-17-7-5-15-13-1...,303.534,304.299793,179.266,0.765793,NaN,NaN,...,54,57,57,57,15,15,15,49,49,49
3,3-beta-dimethylamino-20-beta-hydroxy-5-pregnene,1,Steroidal alkaloid,InChI=1S/C23H39NO/c1-15(25)19-8-9-20-18-7-6-16...,345.571,346.310303,134.484,0.739303,InChI=1S/C19H36O4/c1-3-4-5-6-7-8-9-10-11-12-13...,NaN,...,24,48,48,48,68,68,68,21,21,21
4,3-beta-methylamino-20-oxo-5-pregnene,1,Steroidal alkaloid,InChI=1S/C22H35NO/c1-14(24)18-7-8-19-17-6-5-15...,329.528,330.279036,133.517,0.751036,,,...,34,7,7,7,52,52,52,15,15,15
5,Alchorneine,1,Imidazopyrimidine alkaloid,"InChI=1S/C12H19N3O/c1-9(2)10-8-14-7-6-12(3,4)1...",221.304,222.159760,92.382,0.855760,NaN,CNNCc1ccc(cc1)C(=O)NC(C)C,...,21,36,36,36,22,22,22,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Unreported 645,0,MIA,NaN,NaN,645.377500,1491.047,NaN,InChI=1S/C35H58O9/c1-19(2)32-24(7)27(36)18-35(...,CO[C@H]1/C=C/C=C(C)/C[C@H](C)[C@H](O)[C@H](C)\...,...,8,56,56,56,13,13,13,11,11,11
93,Unreported 647,0,MIA,NaN,NaN,647.356805,809.958,NaN,InChI=1S/C40H46N4O4/c1-4-37-13-14-40-24(12-16-...,[H][C@]12[C@]34NC5=CC=C([C@]6([H])N(CC7)[C@@](...,...,32,12,12,12,17,17,17,12,12,12
94,Unreported 703,0,MIA,NaN,NaN,703.422791,760.470,NaN,NaN,NaN,...,28,5,5,5,7,7,7,6,6,6


In [16]:
maxcs = compounds.apply(lambda x: x.loc[["cg", "cs", "ci"]].max(), axis=1).rename("maxcs")
maxtans = compounds.apply(lambda x: x.loc[["tgs", "tgi", "tsi"]].min(), axis=1).rename("maxtans")
maxtans
compounds = compounds.assign(maxcs=maxcs, maxtans=maxtans)
row_value = compounds.apply(lambda x: 1 - x["maxcs"] + 3*x["maxtans"], axis=1)
compounds = compounds.assign(row_value=row_value)
compounds = compounds.sort_values("row_value", ascending=False)
bests = compounds.iloc[:10]
compounds

,Chemical name,Reported,Chemical class,InChI,Relative molecular weight,Precursor m/z,Retention time (sec),Precursor m/z − relative molecular weight,Analog InChI GNPS; peaks ≥ 6; Δ mass ≤ 0.02,Analog Smiles GNPS; peaks ≥ 6; Δ mass ≤ 0.02,...,Ranks Sirius,Rank min ISDB-LOTUS,Rank max ISDB-LOTUS,Rank ISDB-LOTUS,Rank min K,Rank max K,Rank K,maxcs,maxtans,row_value
Id,,,,,,,,,,,,,,,,,,,,,
55,Isorhamnetol,1,Flavonol,InChI=1S/C16H12O7/c1-22-11-4-7(2-3-9(11)18)16-...,316.265,317.065649,138.115,0.800649,InChI=1S/C16H12O7/c1-22-11-4-7(2-3-9(11)18)16-...,O=C1C2=C(O)C=C(O)C=C2OC(C3=CC(OC)=C(O)C=C3)=C1O,...,70,76,76,76,96,96,96,0.804517,0.674419,2.218739
54,Isoquercitrin,1,Flavonol-type flavonoid,InChI=1S/C21H20O12/c22-6-13-15(27)17(29)18(30)...,464.379,465.102948,88.944,0.723948,InChI=1S/C21H20O12/c22-6-13-15(26)17(28)19(30)...,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O...,...,30,88,88,88,79,79,79,0.789319,0.437500,1.523181
19,Conessine,1,Steroidal alkaloid,InChI=1S/C24H40N2/c1-16-20-8-9-22-19-7-6-17-14...,356.598,357.326131,83.810,0.728131,InChI=1S/C24H40N2/c1-16-20-8-9-22-19-7-6-17-14...,C1=C2CC(N(C)C)CCC2(C)C3CCC45CN(C)C(C)C5CCC4C3C1,...,71,47,47,47,62,62,62,0.694019,0.384615,1.459827
44,Holaphylline,1,Steroidal alkaloid,InChI=1S/C22H35NO/c1-14(24)18-7-8-19-17-6-5-15...,329.528,330.279162,137.509,0.751162,,,...,52,54,54,54,27,27,27,0.435046,0.213333,1.204954
91,Unreported 481,0,Antimycine depsipeptide,NaN,NaN,481.218200,275.280,NaN,NaN,NaN,...,24,3,3,3,1,1,1,0.143283,0.111111,1.190050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,Funtumine,1,Steroidal alkaloid,InChI=1S/C21H35NO/c1-13(23)17-6-7-18-16-5-4-14...,317.517,318.278733,133.251,0.761733,"""""""InChI=1S/C17H32O4/c1-3-4-5-6-7-8-9-10-11-12...",O=C(O)C(CCCCCCCCCCCC)C(C)C(=O)O,...,92,82,82,82,51,51,51,0.937950,0.057143,0.233479
18,Conarrhimine,1,Steroidal alkaloid,InChI=1S/C21H34N2/c1-13-17-5-6-19-16-4-3-14-11...,314.517,315.278641,93.997,0.761641,NaN,CCCCCCCCCCCC(NC(CCCN)C(O)=O)=O,...,88,59,59,59,32,32,32,0.843438,0.011905,0.192276
7,Ancistroealaine A,1,Naphthalene-Isoquinoline alkaloid,InChI=1S/C26H29NO4/c1-14-10-18-17(8-9-20(28-4)...,419.521,420.217224,113.534,0.696224,InChI=1S/C22H26O7/c1-6-7-22-10-16(25-4)17(23)2...,NaN,...,93,56,56,56,46,46,46,0.946511,0.043956,0.185357


In [17]:
indicators.loc[bests.index]

,cg,cs,ci,tgs,tgi,tsi,Rank K
Id,,,,,,,
55,0.804517,0.679773,0.472981,0.674419,1.000000,0.674419,96
54,0.789319,0.203771,0.513674,0.437500,0.550725,0.576271,79
19,0.646152,0.694019,0.402314,0.444444,0.516667,0.384615,62
44,0.423630,0.435046,0.414101,0.229730,0.213333,0.551724,27
91,0.045815,0.143283,0.105071,0.175926,0.116071,0.111111,1
53,0.780872,0.261866,0.363611,0.267606,0.370370,0.323529,76
23,0.154955,0.070621,0.187690,0.112069,0.112782,0.067227,3
96,0.221073,0.273088,0.218319,0.091837,0.088000,0.109756,5
93,0.356873,0.045456,0.291762,0.101351,0.169935,0.111111,12


In [34]:
ranks.loc[[19] + list(range(91, 97))]

,Rank min GNPS original,Rank max GNPS original,Ranks GNPS original,Rank min GNPS iterated,Rank max GNPS iterated,Rank GNPS iterated,Rank min Sirius,Rank max Sirius,Ranks Sirius,Rank min ISDB-LOTUS,Rank max ISDB-LOTUS,Rank ISDB-LOTUS,Rank min K,Rank max K,Rank K
Id,,,,,,,,,,,,,,,
19,58,58,58,58,58,58,71,71,71,47,47,47,62,62,62
91,1,3,1–3,2,2,2,24,24,24,3,3,3,1,1,1
92,11,11,11,8,8,8,56,56,56,13,13,13,11,11,11
93,37,37,37,32,32,32,12,12,12,17,17,17,12,12,12
94,1,3,1–3,28,28,28,5,5,5,7,7,7,6,6,6
95,6,6,6,3,3,3,73,73,73,4,4,4,26,26,26
96,18,18,18,14,14,14,34,34,34,9,9,9,5,5,5
